In [1]:
!pip install Keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications import densenet
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras import regularizers
from keras import backend as K
import sys
from matplotlib import pyplot
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.optimizers import SGD
from keras.utils import multi_gpu_model
# import tensorflow as tf
# from keras.backend.tensorflow_backend import set_session
from keras.callbacks import ModelCheckpoint
import pandas as pd
from keras.layers.normalization import BatchNormalization
# from tensorflow.keras.callbacks import TensorBoard
import time
import keras

You should consider upgrading via the 'pip install --upgrade pip' command.


Using TensorFlow backend.


In [2]:
# import tensorflow as tf
# import keras.backend.tensorflow_backend as ktf


# def get_session(gpu_fraction=0.5):
#     gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction,
#                                 allow_growth=True)
#     return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))


# ktf.set_session(get_session())

In [2]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [66]:
K.set_learning_phase(1)

In [3]:
img_width, img_height = 224, 224
nb_train_samples = 539231
nb_validation_samples = 240424
epochs = 10
batch_size = 100
n_classes = 1

In [4]:
train_data_dir = "../../../../../raid/Data/Master_Dataset/mix/train"
validation_data_dir = "../../../../../raid/Data/Master_Dataset/mix/validation"

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode = 'constant',
    cval = 1,
    rotation_range = 5,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')



Found 539231 images belonging to 2 classes.
Found 240424 images belonging to 2 classes.


In [4]:
for i, l in validation_generator:
    print(l)

[0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1.
 0. 0. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1.
 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1.
 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 1.
 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1.
 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1.
 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0.
 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0.
 1. 1. 0. 1. 1. 0. 0. 0.]
[0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1.
 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1.
 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1.
 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 1. 0. 1. 0.
 0. 1. 1. 0. 1. 1. 0. 0. 

[0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1.
 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0.
 0. 0. 1. 0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 1.
 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0.
 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1.
 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1.
 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 1. 1.
 0. 1. 1. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 1.
 0. 1. 0. 0. 1. 0. 0. 1.]
[0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0.
 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0.
 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 0. 1. 0. 0. 1.
 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1.
 0. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0.
 1. 0. 0. 1. 1. 1. 0. 0. 

KeyboardInterrupt: 

In [5]:
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.Session(config=config)
set_session(sess) 

In [9]:
config =  tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.Session(config=config)
tf.keras.backend.set_session(sess)  

NameError: name 'tf' is not defined

In [6]:
# def build_model():
#     base_model = densenet.DenseNet121(input_shape=(img_width, img_height, 3),
#                                      weights="imagenet_models/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5",
#                                      include_top=False,
#                                      pooling='avg')
   
#     for layer in base_model.layers:
#         layer.trainable = True
#     x=base_model.output
#     x = Dense(1000, kernel_regularizer=regularizers.l1_l2(0.01), activity_regularizer=regularizers.l2(0.01))(x)
#     x = Activation('relu')(x)
#     x = Dense(500, kernel_regularizer=regularizers.l1_l2(0.01), activity_regularizer=regularizers.l2(0.01))(x)
#     x = Activation('relu')(x)
#     predictions = Dense(n_classes, activation='sigmoid')(x)
#     model = Model(inputs=base_model.input, outputs=predictions)
    
#     return model

In [6]:
base_model = densenet.DenseNet121(input_shape=(img_width, img_height, 3),
                                     weights="densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5",
                                     include_top=False,
                                     pooling='avg')

for layer in base_model.layers:
    layer.trainable = True

x = base_model.output
x = Dense(n_classes, activation='sigmoid', name = 'prediction')(x)
model = Model(inputs = base_model.input, outputs = x)



Instructions for updating:
Colocations handled automatically by placer.


In [7]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto',min_delta=1e-2, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=1, mode='auto',verbose=1, min_delta=1e-2, min_lr=0.00001)

In [8]:
i = 0
class LossHistory(Callback):
    def on_epoch_end(self, epoch, logs=None):
        global i
        self.model.save("paper/weights_of_epoch_" + str(i) + ".hdf5")
#         summarize_diagnostics(self.model_history, i)
        i += 1
        

In [6]:
i = 1
for layers in model.layers:
    print(i, layers, layers.trainable)
    i += 1

1 <keras.engine.input_layer.InputLayer object at 0x7f9c0dcf60b8> False
2 <keras.layers.convolutional.ZeroPadding2D object at 0x7f9bf4774240> False
3 <keras.layers.convolutional.Conv2D object at 0x7f9bf4774358> False
4 <keras.layers.normalization.BatchNormalization object at 0x7f9bf47747b8> False
5 <keras.layers.core.Activation object at 0x7f9bf4774e48> False
6 <keras.layers.convolutional.ZeroPadding2D object at 0x7f9bf4774be0> False
7 <keras.layers.pooling.MaxPooling2D object at 0x7f9bf4774748> False
8 <keras.layers.normalization.BatchNormalization object at 0x7f9bf4721c50> False
9 <keras.layers.core.Activation object at 0x7f9bedb17d30> False
10 <keras.layers.convolutional.Conv2D object at 0x7f9bedb32c18> False
11 <keras.layers.normalization.BatchNormalization object at 0x7f9bedac47f0> False
12 <keras.layers.core.Activation object at 0x7f9bedaa55f8> False
13 <keras.layers.convolutional.Conv2D object at 0x7f9beda5df60> False
14 <keras.layers.merge.Concatenate object at 0x7f9bed9eb550> F

In [5]:
# summary 
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
____________________________________________________________________________________________

In [9]:
e = 10
# model = tf.keras.models.load_model('/home/vision/mrunal/detect_deepfake/densenet121/paper/weights_of_epoch_0.hdf5')
sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model = multi_gpu_model(model, gpus=2)
model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
final_location = []
lh = LossHistory()
# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [lh, early_stop, reduce_lr]

model_history = model.fit_generator(
train_generator,
epochs=e,
steps_per_epoch = nb_train_samples // train_generator.batch_size,
validation_data=validation_generator,
verbose=1,
validation_steps=nb_validation_samples // validation_generator.batch_size,
callbacks=callbacks_list)

#model.save("weights_best_" + str(e) + ".hdf5")

# _,acc=model.evaluate_generator(validation_generator, steps=nb_validation_samples, max_queue_size=10,verbose=1, workers=1, use_multiprocessing=False)
# print('> %.3f' % (acc * 100.0))

loss=model_history.history['loss']
acc=model_history.history['accuracy']
valacc=model_history.history['val_accuracy']
valloss=model_history.history['val_loss']
location = [e,loss,acc,valacc, valloss]
final_location.append(location)
save = pd.DataFrame(final_location,columns=['epochs','loss','acc','valacc','valloss'])
save.to_csv('/home/vision/mrunal/detect_deepfake/densenet121/paper/densenet_log_'+str(e)+'.csv')

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
5392/5392 [==============================] - 7986s 1s/step - loss: 0.1242 - accuracy: 0.9480 - val_loss: 0.4775 - val_accuracy: 0.8698
Epoch 2/10
5392/5392 [==============================] - 7788s 1s/step - loss: 0.0488 - accuracy: 0.9807 - val_loss: 0.3897 - val_accuracy: 0.8513

Epoch 00002: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 3/10
5392/5392 [==============================] - 7153s 1s/step - loss: 0.0301 - accuracy: 0.9884 - val_loss: 0.3937 - val_accuracy: 0.8819
Epoch 4/10
5392/5392 [==============================] - 7007s 1s/step - loss: 0.0272 - accuracy: 0.9893 - val_loss: 0.4800 - val_accuracy: 0.8827

Epoch 00004: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.
Epoch 5/10
5392/5392 [==============================] - 6853s 1s/step - loss: 0.0249 - accuracy: 0.9903 - val_loss: 0.5188 - val_accuracy: 0.8909
Restoring model weights from the end of the best epoch

Ep

In [87]:
# again interrupted at epoch 1
e = 9
final_location = []
# model = tf.keras.models.load_model('/home/vision/mrunal/detect_deepfake/densenet121/paper/weights_of_epoch_0.hdf5')
model = keras.models.load_model("/home/vision/mrunal/detect_deepfake/densenet121/paper/weights_of_epoch_1.hdf5")
sgd = SGD(lr=0.0005, decay=1e-6, momentum=0.9, nesterov=True)
# model = multi_gpu_model(model, gpus=2)
model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])

lh = LossHistory()
# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [lh, early_stop, reduce_lr]

model_history = model.fit_generator(
train_generator,
epochs=e,
steps_per_epoch = nb_train_samples // train_generator.batch_size,
validation_data=validation_generator,
verbose=1,
validation_steps=nb_validation_samples // validation_generator.batch_size,
callbacks=callbacks_list)

#model.save("weights_best_" + str(e) + ".hdf5")

# _,acc=model.evaluate_generator(validation_generator, steps=nb_validation_samples, max_queue_size=10,verbose=1, workers=1, use_multiprocessing=False)
# print('> %.3f' % (acc * 100.0))

loss=model_history.history['loss']
acc=model_history.history['accuracy']
valacc=model_history.history['val_accuracy']
valloss=model_history.history['val_loss']
location = [e,loss,acc,valacc, valloss]
final_location.append(location)
save = pd.DataFrame(final_location,columns=['epochs','loss','acc','valacc','valloss'])
save.to_csv('/home/vision/mrunal/detect_deepfake/densenet121/paper/densenet_log_'+str(e)+'.csv')

Epoch 1/9
2696/2696 [==============================] - 7045s 3s/step - loss: 0.5798 - accuracy: 0.6944 - val_loss: 0.5675 - val_accuracy: 0.6451
Epoch 2/9
2696/2696 [==============================] - 6649s 2s/step - loss: 0.5789 - accuracy: 0.6950 - val_loss: 0.5998 - val_accuracy: 0.6562
Epoch 3/9
2696/2696 [==============================] - 6850s 3s/step - loss: 0.5788 - accuracy: 0.6960 - val_loss: 0.5894 - val_accuracy: 0.6533

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 4/9
2696/2696 [==============================] - 6774s 3s/step - loss: 0.5787 - accuracy: 0.6959 - val_loss: 0.6611 - val_accuracy: 0.6499
Restoring model weights from the end of the best epoch

Epoch 00004: ReduceLROnPlateau reducing learning rate to 2.0000000949949027e-05.
Epoch 00004: early stopping


In [84]:
print(i)

2


In [76]:
model.save("/home/vision/mrunal/detect_deepfake/densenet121/paper/weights_of_epoch_0.hdf5")

In [77]:
#loaded without error
model = keras.models.load_model("/home/vision/mrunal/detect_deepfake/densenet121/paper/weights_of_epoch_0.hdf5")

In [78]:
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
lambda_5 (Lambda)               (None, 224, 224, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
lambda_6 (Lambda)               (None, 224, 224, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
model_7 (Model)                 (None, 1)            7038529     lambda_5[0][0]                   
                                                                 lambda_6[0][0]             

In [63]:
model = tf.keras.models.load_model('/home/vision/mrunal/detect_deepfake/densenet121/paper/weights_of_epoch_0.hdf5')

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


AttributeError: module 'tensorflow.python.keras.backend' has no attribute 'slice'

In [53]:
# interrupted at epoch 1

c = [LossHistory(), early_stop, reduce_lr]
model.evaluate_generator(validation_generator, steps=nb_validation_samples // validation_generator.batch_size, callbacks=c, workers=1, use_multiprocessing=False, verbose=1)


1202/1202 [==============================] - 1040s 866ms/step


[0.7072643041610718, 0.5397753715515137]

In [7]:
for layers in model.layers:
    print(layers, layers.trainable)

<keras.engine.input_layer.InputLayer object at 0x7fe6606fe3c8> False
<keras.layers.convolutional.ZeroPadding2D object at 0x7fe1083ffc50> False
<keras.layers.convolutional.Conv2D object at 0x7fe1083ffe10> False
<keras.layers.normalization.BatchNormalization object at 0x7fe108424160> False
<keras.layers.core.Activation object at 0x7fe108424e10> False
<keras.layers.convolutional.ZeroPadding2D object at 0x7fe1082c7e80> False
<keras.layers.pooling.MaxPooling2D object at 0x7fe10833f400> False
<keras.layers.normalization.BatchNormalization object at 0x7fe1084375c0> False
<keras.layers.core.Activation object at 0x7fe1085b6b00> False
<keras.layers.convolutional.Conv2D object at 0x7fe1086d9e48> False
<keras.layers.normalization.BatchNormalization object at 0x7fe10870a3c8> False
<keras.layers.core.Activation object at 0x7fe108503a90> False
<keras.layers.convolutional.Conv2D object at 0x7fe108765ac8> False
<keras.layers.merge.Concatenate object at 0x7fe1080ad8d0> False
<keras.layers.normalization.

In [23]:
# layer = model.layers[428]
# layer.trainable = True
false = 0
true = 0
for layers in model.layers:
    if(layers.trainable == False):
        false += 1
    else:
        true += 1
    

In [24]:
print(false, true)

428 2


In [22]:
print(len(model.layers))

430


In [25]:
l = 1
for layers in model.layers:
    print(layers, layers.trainable)

<keras.engine.input_layer.InputLayer object at 0x7f09c36eff28> False
<keras.layers.convolutional.ZeroPadding2D object at 0x7f06043ed470> False
<keras.layers.convolutional.Conv2D object at 0x7f06043ed630> False
<keras.layers.normalization.BatchNormalization object at 0x7f060440d7f0> False
<keras.layers.core.Activation object at 0x7f060440d160> False
<keras.layers.convolutional.ZeroPadding2D object at 0x7f0644604080> False
<keras.layers.pooling.MaxPooling2D object at 0x7f0604465320> False
<keras.layers.normalization.BatchNormalization object at 0x7f0804630208> False
<keras.layers.core.Activation object at 0x7f062c0b3630> False
<keras.layers.convolutional.Conv2D object at 0x7f0604712c18> False
<keras.layers.normalization.BatchNormalization object at 0x7f060466d940> False
<keras.layers.core.Activation object at 0x7f06046ea8d0> False
<keras.layers.convolutional.Conv2D object at 0x7f062c05f940> False
<keras.layers.merge.Concatenate object at 0x7f0604156208> False
<keras.layers.normalization.

In [16]:
print(model.layers[427])

In [7]:
print(len(base_model.layers))

428


In [7]:
model=build_model()
model = multi_gpu_model(model, gpus=2)


Instructions for updating:
Colocations handled automatically by placer.


In [40]:
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
lambda_13 (Lambda)              (None, 224, 224, 3)  0           input_9[0][0]                    
__________________________________________________________________________________________________
lambda_14 (Lambda)              (None, 224, 224, 3)  0           input_9[0][0]                    
__________________________________________________________________________________________________
lambda_15 (Lambda)              (None, 224, 224, 3)  0           input_9[0][0]                    
____________________________________________________________________________________________

In [7]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto',min_delta=1e-2, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=1, mode='auto',verbose=1, min_delta=1e-2, min_lr=0.00001)

In [8]:
i = 0
class LossHistory(Callback):
    def on_epoch_end(self, epoch, logs=None):
        global i
        self.model.save("paper/weights_of_epoch_" + str(i) + ".hdf5")
#         summarize_diagnostics(self.model_history, i)
        i += 1
        

In [79]:
print(i)

0


In [73]:
def summarize_diagnostics(history, i):
	# plot loss
	pyplot.subplot(121)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(122)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_acc'], color='orange', label='test')
	# save plot to file
	filename = "/home/vision/mrunal/detect_deepfake/densenet121/paper/plot_of_epoch{}".format(i)
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()

In [14]:
y = 1
base_model = densenet.DenseNet121(input_shape=(img_width, img_height, 3),
                                     weights="densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5",
                                     include_top=False,
                                     pooling='avg')


for layer in base_model.layers:
    layer.trainable = True


num_classes = 1
e=4

new_model = Model(inputs = base_model.input, outputs = base_model.layers[-2].output)
x = new_model.output

dense_layers = [0, 1, 2, 3]
layer_sizes = [0, 32, 64, 128]
conv_layers = [0, 1, 2, 3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)
            
            final_location=[]
            location=[]
            
            
            
            for l in range(conv_layer):
                
                if conv_layer is not 0:
                    model = Conv2D(layer_size, (3, 3), activation = 'relu')(x)

                    model = MaxPooling2D(pool_size=(2, 2))(model)

            if(conv_layer == 0):
                
                model = Flatten()(x)
            else:
                model = Flatten()(model)
                

            for _ in range(dense_layer):
            
                model = Dense(layer_size, activation = 'relu')(model)


            model = Dense(1, activation = 'sigmoid')(model)

            

            head_model = Model(inputs = base_model.input, outputs = model)
            head_model.summary()
            
            sgd = SGD(lr=0.001, decay=1e-5, momentum=0.9, nesterov=True)
    
            model = multi_gpu_model(head_model, gpus=2)
        
            model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])

            filepath="weights_best_" + str(y)+ '_'+ str(e) + ".hdf5"
            
            checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
            
            callbacks_list = [checkpoint,early_stop, reduce_lr]

            model_history = model.fit_generator(
            train_generator,
            epochs=e,
            steps_per_epoch=nb_train_samples // batch_size,
            validation_data=validation_generator,
            verbose=1,
            validation_steps=nb_validation_samples // batch_size,
            callbacks=callbacks_list)
            
            y = y + 1
            
            summarize_diagnostics(model_history)
            loss=model_history.history['loss']
            acc=model_history.history['accuracy']
            valacc=model_history.history['val_accuracy']
            valloss=model_history.history['val_loss']
            location = [e,loss,acc,valacc, valloss]
            final_location.append(location)
            save = pd.DataFrame(final_location,columns=['epochs','loss','acc','valacc','valloss'])
            save.to_csv('densenet_log_'+str(e)+'.csv')

0-conv-0-nodes-0-dense-1577895819
Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_3 (ZeroPadding2D (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_3[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
__________________________________________________________

Epoch 1/4
 173/3594 [>.............................] - ETA: 1:49:36 - loss: 0.9160 - accuracy: 0.7708

ResourceExhaustedError: OOM when allocating tensor with shape[75,992,7,7] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node replica_0_1/model_5/conv5_block16_0_bn/FusedBatchNormV3}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node training_1/SGD/gradients/dense_2_1/concat_grad/Slice_1}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [17]:
def define_model():
    base_model = densenet.DenseNet121(input_shape=(img_width, img_height, 3),
                                     weights="densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5",
                                     include_top=False,
                                     pooling='avg')
    for layer in base_model.layers:
        layer.trainable = True
    
    
    new_model = build_model(hp)
    
    head_model = Model(inputs = base_model.input, outputs = new_model.output)
    
    return head_model

In [ ]:
def build_model(hp):
    base_model = densenet.DenseNet121(input_shape=(img_width, img_height, 3),
                                     weights="densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5",
                                     include_top=False,
                                     pooling='avg')
    for layer in base_model.layers:
        layer.trainable = True
        
    x = base_model.output
    
    new_model = Model(inputs = base_model.input, outputs = x)
    
    for i in range(hp.Int('num_layers', 0, 3)):
        new_model = layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu')(x)
    
    sgd = SGD(lr=0.0005, decay=1e-6, momentum=0.9, nesterov=True)
    
    model = multi_gpu_model(new_model, gpus=2)
    
    
    
    model.compile(optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])), loss='binary_crossentropy', metrics=['accuracy'])
    
    
    return model

In [47]:
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5,
    executions_per_trial=3,
    directory='log_dir_1',
    project_name='hyperparameter_search')


Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/kerastuner/engine/hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-45-420e4513ccef>", line 22, in build_model
    model = multi_gpu_model(new_model, gpus=2)
  File "/usr/local/lib/python3.5/dist-packages/keras/utils/multi_gpu_utils.py", line 150, in multi_gpu_model
    available_devices = _get_available_devices()
  File "/usr/local/lib/python3.5/dist-packages/keras/utils/multi_gpu_utils.py", line 16, in _get_available_devices
    return K.tensorflow_backend._get_available_gpus() + ['/cpu:0']
  File "/usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py", line 506, in _get_available_gpus
    _LOCAL_DEVICES = tf.config.experimental_list_devices()
AttributeError: module 'tensorflow_core._api.v2.config' has no attribute 'experimental_list_devices'


Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/kerastuner/engine/hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-45-420e4513ccef>", line 22, in build_model
    model = multi_gpu_model(new_model, gpus=2)
  File "/usr/local/lib/python3.5/dist-packages/keras/utils/multi_gpu_utils.py", line 150, in multi_gpu_model
    available_devices = _get_available_devices()
  File "/usr/local/lib/python3.5/dist-packages/keras/utils/multi_gpu_utils.py", line 16, in _get_available_devices
    return K.tensorflow_backend._get_available_gpus() + ['/cpu:0']
  File "/usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py", line 506, in _get_available_gpus
    _LOCAL_DEVICES = tf.config.experimental_list_devices()
AttributeError: module 'tensorflow_core._api.v2.config' has no attribute 'experimental_list_devices'


Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/kerastuner/engine/hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-45-420e4513ccef>", line 22, in build_model
    model = multi_gpu_model(new_model, gpus=2)
  File "/usr/local/lib/python3.5/dist-packages/keras/utils/multi_gpu_utils.py", line 150, in multi_gpu_model
    available_devices = _get_available_devices()
  File "/usr/local/lib/python3.5/dist-packages/keras/utils/multi_gpu_utils.py", line 16, in _get_available_devices
    return K.tensorflow_backend._get_available_gpus() + ['/cpu:0']
  File "/usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py", line 506, in _get_available_gpus
    _LOCAL_DEVICES = tf.config.experimental_list_devices()
AttributeError: module 'tensorflow_core._api.v2.config' has no attribute 'experimental_list_devices'


Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/kerastuner/engine/hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-45-420e4513ccef>", line 22, in build_model
    model = multi_gpu_model(new_model, gpus=2)
  File "/usr/local/lib/python3.5/dist-packages/keras/utils/multi_gpu_utils.py", line 150, in multi_gpu_model
    available_devices = _get_available_devices()
  File "/usr/local/lib/python3.5/dist-packages/keras/utils/multi_gpu_utils.py", line 16, in _get_available_devices
    return K.tensorflow_backend._get_available_gpus() + ['/cpu:0']
  File "/usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py", line 506, in _get_available_gpus
    _LOCAL_DEVICES = tf.config.experimental_list_devices()
AttributeError: module 'tensorflow_core._api.v2.config' has no attribute 'experimental_list_devices'


Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/kerastuner/engine/hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-45-420e4513ccef>", line 22, in build_model
    model = multi_gpu_model(new_model, gpus=2)
  File "/usr/local/lib/python3.5/dist-packages/keras/utils/multi_gpu_utils.py", line 150, in multi_gpu_model
    available_devices = _get_available_devices()
  File "/usr/local/lib/python3.5/dist-packages/keras/utils/multi_gpu_utils.py", line 16, in _get_available_devices
    return K.tensorflow_backend._get_available_gpus() + ['/cpu:0']
  File "/usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py", line 506, in _get_available_gpus
    _LOCAL_DEVICES = tf.config.experimental_list_devices()
AttributeError: module 'tensorflow_core._api.v2.config' has no attribute 'experimental_list_devices'


Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/kerastuner/engine/hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-45-420e4513ccef>", line 22, in build_model
    model = multi_gpu_model(new_model, gpus=2)
  File "/usr/local/lib/python3.5/dist-packages/keras/utils/multi_gpu_utils.py", line 150, in multi_gpu_model
    available_devices = _get_available_devices()
  File "/usr/local/lib/python3.5/dist-packages/keras/utils/multi_gpu_utils.py", line 16, in _get_available_devices
    return K.tensorflow_backend._get_available_gpus() + ['/cpu:0']
  File "/usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py", line 506, in _get_available_gpus
    _LOCAL_DEVICES = tf.config.experimental_list_devices()
AttributeError: module 'tensorflow_core._api.v2.config' has no attribute 'experimental_list_devices'


RuntimeError: Too many failed attempts to build model.

In [ ]:
tuner.search_space_summary()

In [ ]:
tuner.search( training_data = train_generator
             epochs=5,
             validation_data= validation_generator)

In [ ]:
models = tuner.get_best_models(num_models=2)

In [ ]:
tuner.results_summary()

In [21]:
# for layer in base_model.layers:
#         layer.trainable = True
    

# #number of classes in your dataset e.g. 20
# num_classes = 1

# x = Dense(1024, activation='relu')(base_model.output)
# x = Dropout(0.5)(x)
# x = BatchNormalization()(x)
# predictions = Dense(num_classes, activation = 'sigmoid')(x)

# #create graph of your new model
# head_model = Model(inputs = base_model.input, outputs = predictions)
# head_model.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_3 (ZeroPadding2D (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_3[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
____________________________________________________________________________________________

In [ ]:
epochs=[6,10,14]
x=1
for e in epochs:
    final_location=[]
    location=[]
    #base_model=Sequential()
    model=define_model()
    
    sgd = SGD(lr=0.0005, decay=1e-6, momentum=0.9, nesterov=True)
    model = multi_gpu_model(model, gpus=2)
    
    
    
    model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
    
    filepath="weights_best_" + str(e) + ".hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint,early_stop, reduce_lr]
    
    model_history = model.fit_generator(
    train_generator,
    epochs=e,
    steps_per_epoch = nb_train_samples // train_generator.batch_size,
    validation_data=validation_generator,
    verbose=1,
    validation_steps=nb_validation_samples // validation_generator.batch_size,
    callbacks=callbacks_list)
    
    #model.save("weights_best_" + str(e) + ".hdf5")
    
    _,acc=model.evaluate_generator(validation_generator, steps=nb_validation_samples, max_queue_size=10,verbose=1, workers=1, use_multiprocessing=False)
    print('> %.3f' % (acc * 100.0))
    summarize_diagnostics(model_history)
    
    loss=model_history.history['loss']
    acc=model_history.history['accuracy']
    valacc=model_history.history['val_accuracy']
    valloss=model_history.history['val_loss']
    location = [e,loss,acc,valacc, valloss]
    final_location.append(location)
    save = pd.DataFrame(final_location,columns=['epochs','loss','acc','valacc','valloss'])
    save.to_csv('densenet_log_'+str(e)+'.csv')

Instructions for updating:
Use tf.cast instead.
Epoch 1/6
3594/3594 [==============================] - 7075s 2s/step - loss: 0.1946 - accuracy: 0.9155 - val_loss: 0.6307 - val_accuracy: 0.8704

Epoch 00001: val_accuracy improved from -inf to 0.87042, saving model to weights_best_6.hdf5
Epoch 2/6
3594/3594 [==============================] - 7106s 2s/step - loss: 0.0793 - accuracy: 0.9688 - val_loss: 0.5333 - val_accuracy: 0.8746

Epoch 00002: val_accuracy improved from 0.87042 to 0.87457, saving model to weights_best_6.hdf5
Epoch 3/6
3594/3594 [==============================] - 7071s 2s/step - loss: 0.0562 - accuracy: 0.9780 - val_loss: 0.2949 - val_accuracy: 0.8818

Epoch 00003: val_accuracy improved from 0.87457 to 0.88181, saving model to weights_best_6.hdf5
Epoch 4/6
3594/3594 [==============================] - 7100s 2s/step - loss: 0.0450 - accuracy: 0.9824 - val_loss: 0.4008 - val_accuracy: 0.8721

Epoch 00004: val_accuracy did not improve from 0.88181

Epoch 00004: ReduceLROnPlat